In [119]:
from pandas import read_csv
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


In [120]:
import pandas as pd
from data import citylearn_challenge_2022_phase_1 as competition_data
import os.path as osp

solargen_data_path = osp.join(osp.dirname(competition_data.__file__), "solar_data.csv")
solargen_data = pd.read_csv(solargen_data_path)

# for i in range(5):
#     plt.plot(solargen_data["non_shiftable_load"][i*8760 : 8760 + i*8760])
#     plt.show()
solargen_data

,month,day_type,hour,outdoor_dry_bulb,outdoor_dry_bulb_temperature_predicted_6h,outdoor_dry_bulb_temperature_predicted_12h,outdoor_dry_bulb_temperature_predicted_24h,outdoor_relative_humidity,outdoor_relative_humidity_predicted_6h,outdoor_relative_humidity_predicted_12h,...,direct_solar_irradiance_predicted_6h,direct_solar_irradiance_predicted_12h,direct_solar_irradiance_predicted_24h,carbon_intensity,solar_generation,electricity_pricing,electricity_pricing_predicted_6h,electricity_pricing_predicted_12h,electricity_pricing_predicted_24h,solar_generation_future
0,8.0,1.0,1.0,20.1,19.4,22.8,19.4,79.0,79.0,71.0,...,444.0,747.0,0.0,0.157319,0.000000,0.22,0.22,0.22,0.22,0.000000
1,8.0,1.0,2.0,19.7,21.1,22.2,19.4,78.0,73.0,73.0,...,592.0,291.0,0.0,0.154503,0.000000,0.22,0.22,0.22,0.22,0.000000
2,8.0,1.0,3.0,19.3,22.2,22.8,19.4,78.0,71.0,71.0,...,491.0,153.0,0.0,0.150945,0.000000,0.22,0.22,0.22,0.22,0.000000
3,8.0,1.0,4.0,18.9,21.7,22.2,18.9,78.0,73.0,71.0,...,734.0,174.0,0.0,0.151820,0.000000,0.22,0.22,0.54,0.22,0.000000
4,8.0,1.0,5.0,18.3,22.2,21.1,18.3,81.0,71.0,76.0,...,811.0,396.0,0.0,0.158088,0.000000,0.22,0.22,0.54,0.22,0.058017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43785,7.0,1.0,18.0,21.1,20.0,18.3,20.6,81.0,84.0,81.0,...,0.0,100.0,320.0,0.225739,0.124717,0.54,0.22,0.22,0.54,0.016283
43786,7.0,1.0,19.0,20.6,20.1,19.4,20.6,84.0,79.0,79.0,...,0.0,444.0,143.0,0.222553,0.016283,0.54,0.22,0.22,0.54,0.000000
43787,7.0,1.0,20.0,21.1,19.7,21.1,20.0,81.0,78.0,73.0,...,0.0,592.0,0.0,0.216094,0.000000,0.54,0.22,0.22,0.54,0.000000
43788,7.0,1.0,21.0,21.7,19.3,22.2,20.6,79.0,78.0,71.0,...,0.0,491.0,0.0,0.215501,0.000000,0.22,0.22,0.22,0.22,0.000000


In [121]:
#"month", "hour", "outdoor_dry_bulb", "outdoor_dry_bulb_temperature_predicted_6h", "outdoor_relative_humidity", "outdoor_relative_humidity_predicted_6h", "diffuse_solar_irradiance", "diffuse_solar_irradiance_predicted_6h", "diffuse_solar_irradiance_predicted_24h", "direct_solar_irradiance", "direct_solar_irradiance_predicted_6h", "direct_solar_irradiance_predicted_24h", "electricity_pricing", "electricity_pricing_predicted_6h", "electricity_pricing_predicted_24h"

selected_params = ["solar_generation", "direct_solar_irradiance_predicted_24h", "diffuse_solar_irradiance", "solar_generation", "month", "outdoor_dry_bulb", "outdoor_relative_humidity", "outdoor_dry_bulb_temperature_predicted_6h", "direct_solar_irradiance_predicted_24h", "hour", "diffuse_solar_irradiance_predicted_6h", "outdoor_relative_humidity_predicted_6h", "direct_solar_irradiance", "direct_solar_irradiance_predicted_6h", "solar_generation_future"]
solargen_data = solargen_data[selected_params]

In [122]:
solargen_data

,solar_generation,direct_solar_irradiance_predicted_24h,diffuse_solar_irradiance,solar_generation,month,outdoor_dry_bulb,outdoor_relative_humidity,outdoor_dry_bulb_temperature_predicted_6h,direct_solar_irradiance_predicted_24h,hour,diffuse_solar_irradiance_predicted_6h,outdoor_relative_humidity_predicted_6h,direct_solar_irradiance,direct_solar_irradiance_predicted_6h,solar_generation_future
0,0.000000,0.0,0.0,0.000000,8.0,20.1,79.0,19.4,0.0,1.0,201.0,79.0,0.0,444.0,0.000000
1,0.000000,0.0,0.0,0.000000,8.0,19.7,78.0,21.1,0.0,2.0,420.0,73.0,0.0,592.0,0.000000
2,0.000000,0.0,0.0,0.000000,8.0,19.3,78.0,22.2,0.0,3.0,554.0,71.0,0.0,491.0,0.000000
3,0.000000,0.0,0.0,0.000000,8.0,18.9,78.0,21.7,0.0,4.0,778.0,73.0,0.0,734.0,0.000000
4,0.000000,0.0,0.0,0.000000,8.0,18.3,81.0,22.2,0.0,5.0,903.0,71.0,0.0,811.0,0.058017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43785,0.124717,320.0,218.0,0.124717,7.0,21.1,81.0,20.0,320.0,18.0,0.0,84.0,289.0,0.0,0.016283
43786,0.016283,143.0,26.0,0.016283,7.0,20.6,84.0,20.1,143.0,19.0,0.0,79.0,130.0,0.0,0.000000
43787,0.000000,0.0,0.0,0.000000,7.0,21.1,81.0,19.7,0.0,20.0,0.0,78.0,0.0,0.0,0.000000
43788,0.000000,0.0,0.0,0.000000,7.0,21.7,79.0,19.3,0.0,21.0,0.0,78.0,0.0,0.0,0.000000


In [123]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# solargen_data = solargen_data[0:8760]
df_load_scaled = solargen_data.iloc[: , :-1]
df_load_scaled = sc.fit_transform(df_load_scaled)
print(df_load_scaled.shape)

sc2 = StandardScaler()
df_load_scaled_y = sc2.fit_transform(solargen_data[["solar_generation_future"]])
print(df_load_scaled_y.shape)

(43790, 14)
(43790, 1)


In [124]:
hops = 24
no_records = df_load_scaled_y.shape[0]
no_cols = len(selected_params) - 1
X_train = []
y_train = []
for i in range(hops, no_records):
    X_train.append(df_load_scaled[i-hops:i])
    y_train.append(df_load_scaled_y[i][0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

(43766, 24, 14)
(43766,)


In [125]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

In [126]:
X_train_reshaped.shape

(43766, 24, 14)

In [127]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model=Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(hops,no_cols)))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train_reshaped, y_train, validation_split=0, epochs=100, batch_size=128, shuffle=True, workers=24, use_multiprocessing=True)

Epoch 1/100
342/342 [==============================] - 68s 191ms/step - loss: 0.1587
Epoch 2/100
342/342 [==============================] - 64s 189ms/step - loss: 0.0773
Epoch 3/100
342/342 [==============================] - 65s 189ms/step - loss: 0.0685
Epoch 4/100
342/342 [==============================] - 64s 186ms/step - loss: 0.0628
Epoch 5/100
342/342 [==============================] - 65s 190ms/step - loss: 0.0599
Epoch 6/100
342/342 [==============================] - 64s 188ms/step - loss: 0.0573
Epoch 7/100
342/342 [==============================] - 65s 189ms/step - loss: 0.0535
Epoch 8/100
342/342 [==============================] - 65s 189ms/step - loss: 0.0508
Epoch 9/100
342/342 [==============================] - 64s 188ms/step - loss: 0.0488
Epoch 10/100
342/342 [==============================] - 64s 187ms/step - loss: 0.0464
Epoch 11/100
342/342 [==============================] - 64s 189ms/step - loss: 0.0449
Epoch 12/100
342/342 [==============================] - 64s 187

In [ ]:
from pickle import dump
model.save("solar_generation_model.h5")
dump(sc, open("solar_generation_scaler.pkl", "wb"))
dump(sc2, open("solar_generation_scaler_result.pkl", "wb"))

In [60]:
# units = dimensionality of the output space


def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape,
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(hidden_units=2, dense_units=1, input_shape=(3,1), activation=['relu', 'relu'])